In [1]:
import pandas as pd
# titles = [l.strip() for l in open("hackernews").readlines()]
# df = pd.DataFrame(titles).drop_duplicates()
# df = pd.DataFrame(df[0].tolist(), columns=["title"])

# df = pd.read_csv("./data/hacker_news_sample.csv")["title"].dropna()
# df = pd.DataFrame(df, columns=["title"])[10000:20000]

df = pd.read_csv("./data/hackernews_2.csv")[["title","time"]]
df = df.sort_values(by="time")[-50000:]

df

,title,time
2097206,Linux and Baikal Electronics: Why Postgres Sho...,1681047747
2918804,Twitter Is Blocking Likes and Retweets that Me...,1681048015
999963,Delta Sharing: An Open Protocol for Secure Dat...,1681048040
2701694,The Apple II Age: How the Computer Became Pers...,1681048074
2945776,Google Assistant might be doomed: Division “re...,1681048272
...,...,...
474918,"Detecting Threats on 100k Servers, 1000s of Cl...",1686002294
2700235,Dutch Cabinet wants minimum 15% corporate tax ...,1686002552
3172683,Surf Air Moves Ahead on Going Public via Direc...,1686002603
1396875,Dead Duck Day marks that time a scientist witn...,1686002611


In [2]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer

stopwords_en = stopwords.words('english')
stemmer = EnglishStemmer()


def preprocess(text:str):
    # remove the url at the end
    text = text[:text.find('(')]
    # split text
    text = text.lower()
    words = text.split()
    ret = []
    for word in words:
        # strip whitespaces
        word = word.strip()
        # remove stopwords
        if word in stopwords_en:
            continue
        # regex
        word = re.sub(r'[0-9]+', '', word)
        word = re.sub(r'[^a-z]+', '', word)
        word = re.sub(r'\W', '', word)
        # stemming
        word = stemmer.stem(word)
        # remove words len<3
        if len(word) < 2:
            continue
        ret.append(word)
    return ' '.join(ret)

df["text"] = df["title"].apply(preprocess)
df

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,title,time,text
2097206,Linux and Baikal Electronics: Why Postgres Sho...,1681047747,linux baikal electron postgr follow pat
2918804,Twitter Is Blocking Likes and Retweets that Me...,1681048015,twitter block like retweet mention substac
999963,Delta Sharing: An Open Protocol for Secure Dat...,1681048040,delta share open protocol secur data sharin
2701694,The Apple II Age: How the Computer Became Pers...,1681048074,appl ii age comput becam persona
2945776,Google Assistant might be doomed: Division “re...,1681048272,googl assist might doom divis reorgan focus bar
...,...,...,...
474918,"Detecting Threats on 100k Servers, 1000s of Cl...",1686002294,detect threat server cloud account ks cluster
2700235,Dutch Cabinet wants minimum 15% corporate tax ...,1686002552,dutch cabinet want minimum corpor tax cut tax ...
3172683,Surf Air Moves Ahead on Going Public via Direc...,1686002603,surf air move ahead go public via direct listin
1396875,Dead Duck Day marks that time a scientist witn...,1686002611,dead duck day mark time scientist wit gay duck...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df["text"].tolist())
# vectorized.resize((len(df["text"]),300))
print(vectorized.shape)

X = [
    # "Excellence is a habit, but so is failure (awesomekling.github.io)",
    "Meta releases Intermediate Graphics Library (khronos.org)",
]
X = [preprocess(x) for x in X]
print(X)

test_vec = vectorizer.transform(X)
# test_vec.resize((len(X),300))
print(test_vec, test_vec.shape)

(50000, 30719)
['meta releas intermedi graphic librari']
  (0, 22280)	0.3406997841229697
  (0, 16354)	0.3778297637834426
  (0, 14863)	0.38885660117847826
  (0, 13121)	0.6253655986368806
  (0, 11007)	0.4459559826478034 (1, 30719)


In [4]:
cos_sim = cosine_similarity(vectorized,test_vec)
for i in range(test_vec.shape[0]):
    df["similarity"] = [x[0] for x in cos_sim]

top_k = (
    df.sort_values(by="similarity", ascending=False)
        .head(10)
        .query("similarity > 0")
    )
print(X)
top_k

['meta releas intermedi graphic librari']


,title,time,text,similarity
3988873,Should we take measurements at an intermediate...,1683704878,take measur intermedi design point pdf,0.369048
1310841,Anna's Archive – A Meta Library of Shadow Libr...,1683324231,anna archiv meta librari shadow librari,0.363795
2188024,Building a Compiler with Multi-Level Intermedi...,1683048755,build compil multilevel intermedi represent,0.347657
206023,Satin – A 3D Graphics Framework Built on Apple...,1681119248,satin graphic framework built appl meta,0.316667
3596238,Meta to Release Twitter Clone,1684616608,meta releas twitter clon,0.309401
1659015,Berkeley Graphics,1681491306,berkeley graphic,0.292024
1245995,Wgpu – Portable graphics library for Rust base...,1681233450,wgpu portabl graphic librari rust base webgpu ap,0.267835
115002,WebSocket library for Go – gobwas/ws – Release...,1681114268,websocket librari go gobwasw releas,0.252869
3339928,Matrox Announces LUMA Graphics Cards Powered b...,1682743018,matrox announc luma graphic card power intel a...,0.252413
1093549,How should I manage my data library?,1685355893,manag data librari,0.250844


## LDA

In [5]:
# from sklearn.decomposition import LatentDirichletAllocation

# lda = LatentDirichletAllocation(
#     n_components=2,
#     random_state=0,
# )
# Xvec = lda.fit_transform(vectorized)
# print(Xvec)

In [6]:
# import matplotlib.pyplot as plt
# pltX = [dot[0] for dot in Xvec]
# pltY = [dot[1] for dot in Xvec]

# newX,newY = lda.transform(test_vec)[0]
# print(newX, newY)

# plt.scatter(pltX,pltY)
# plt.scatter(newX,newY,c="red")

## LDA with gensim

In [7]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaModel
from gensim.models.coherencemodel import CoherenceModel

texts = df["text"].str.split()
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
from tqdm import tqdm
models = []
for n in range(20,41,2):
    lda = LdaModel(corpus, num_topics=n)
    coherence = CoherenceModel(model = lda, texts = texts, dictionary = dictionary, coherence = 'c_v')
    models.append((n,lda,coherence,coherence.get_coherence()))
    print(models[-1][0], models[-1][-1])

20 0.43560607804943735
22 0.4520653723882315
24 0.46443991680310054
26 0.4788932609598744
28 0.49466363279927755
30 0.49376786594130484
32 0.5044161397363871
34 0.5155652812768866
36 0.5180163338439137
38 0.5216296823924848
40 0.5399092819684822


In [9]:
import pyLDAvis.gensim
lda = models[-1][1]
print(lda)

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics = False)
pyLDAvis.display(lda_display)

LdaModel<num_terms=30719, num_topics=40, decay=0.5, chunksize=2000>


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy